# Agente para extração de dados de jogos

# Nome: Ryan Henrique Pereira Cardoso

## Necessário criar um arquivo .env com uma api key do mistral antes de usar o agente.
## Necessário também instalar as dependencias no requirements.txt

Este agente extrai os dados vgsales.csv, o qual contém informações sobre vendas de diversos jogos e responde perguntas do usuário. 
O agente está preparado para responder perguntas como:

- Qual o jogo mais vendido 
- Quais os 5 (ou N) jogos mais vendidos 
- Qual o jogo menos vendido 
- Quais os N jogos menos vendidos 
- Qual a soma das vendas dos N jogos mais vendidos 
- Qual a soma das vendas dos N jogos menos vendidos 
- Quantos jogos a publisher X publicou 
- Qual o jogo que mais vendeu no pais X (estados unidos, japao ou europa) 
- Qual o ano com mais jogos lancados 
- Qual a plataforma com mais jogos lancados 
- Qual o genero com mais jogos lancados 
- Qual o ano com menos jogos lancados 
- Qual a plataforma com menos jogos lancados 
- Qual o genero com menos jogos lancados

## Realizar importações

In [2]:
import pandas as pd
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_mistralai import MistralAIEmbeddings
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.tools import tool
from langchain.agents import create_agent

load_dotenv()

True

## CARREGAR DATASET

In [3]:
CSV_PATH = "./datasets/vgsales.csv"
df = pd.read_csv(CSV_PATH)
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")

## Inicializar modelo e embeddings

In [4]:
model = init_chat_model("mistral-small-latest")

emb = MistralAIEmbeddings(model="mistral-embed")
vector_store = InMemoryVectorStore(emb)

/workspaces/Atividade_RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Realizar indexação

In [5]:
docs = []
for idx, row in df.iterrows():
    content = (
        f"Rank: {row.get('Rank')}\n"
        f"Name: {row.get('Name')}\n"
        f"Platform: {row.get('Platform')}\n"
        f"Year: {row.get('Year')}\n"
        f"Genre: {row.get('Genre')}\n"
        f"Publisher: {row.get('Publisher')}\n"
        f"NA_Sales: {row.get('NA_Sales')}\n"
        f"EU_Sales: {row.get('EU_Sales')}\n"
        f"JP_Sales: {row.get('JP_Sales')}\n"
        f"Other_Sales: {row.get('Other_Sales')}\n"
        f"Global_Sales: {row.get('Global_Sales')}"
    )
    docs.append(Document(page_content=content, metadata={"index": idx}))

print("Indexando documentos (gerando embeddings)...")
vector_store.add_documents(docs)
print("Indexação concluída.")

Indexando documentos (gerando embeddings)...
Indexação concluída.


## Configurar modelo de extração de intenção

In [11]:
INTENT_PROMPT = """
Você é um classificador de intenções para perguntas sobre um dataset de vendas de videogames.
Analise a pergunta do usuário e retorne um JSON no formato:

{
  "intent": "...",
  "n": null OU número,
  "publisher": null OU string,
  "region": null OU "NA" | "EU" | "JP"
}

RESPONDA APENAS COM UM JSON VÁLIDO E NADA MAIS

Intenções válidas:
- "mais_vendido"
- "menos_vendido"
- "top_n_mais_vendidos"
- "top_n_menos_vendidos"
- "soma_top_n_mais_vendidos"
- "soma_top_n_menos_vendidos"
- "quantidade_por_publisher"
- "mais_vendido_regiao"
- "ano_com_mais_jogos"
- "ano_com_menos_jogos"
- "plataforma_com_mais_jogos"
- "plataforma_com_menos_jogos"
- "genero_com_mais_jogos"
- "genero_com_menos_jogos"
- "buscar_semelhantes" (default quando não entender)

Se a pergunta mencionar "5", "10", "N", etc., coloque o valor em "n".

Identifique região:
- "estados unidos" → "NA"
- "europa" → "EU"
- "japão" → "JP"
"""

intent_model = init_chat_model("mistral-small-latest")

def extrair_intencao(pergunta: str):
    resposta = intent_model.invoke(
        [
            {"role": "system", "content": INTENT_PROMPT},
            {"role": "user", "content": pergunta},
        ]
    )
    import json

    try:
        return json.loads(resposta.content)
    except:
        return {"intent": "buscar_semelhantes", "n": None, "publisher": None, "region": None}

## Configurar tool

In [7]:
@tool
def buscar_semelhantes(query: str):
    """Busca por similaridade usando embeddings."""
    res = vector_store.similarity_search(query, k=5)
    return "\n".join([f"- {d.page_content}" for d in res])

## Definir funções de consulta

In [8]:
def top_n(df, n, asc):
    sorted_df = df.sort_values("Global_Sales", ascending=asc).head(n)
    return sorted_df[["Name", "Platform", "Global_Sales"]].to_string(index=False)


def soma_n(df, n, asc):
    sorted_df = df.sort_values("Global_Sales", ascending=asc).head(n)
    return float(sorted_df["Global_Sales"].sum())


def jogo_unico(df, asc):
    linha = df.sort_values("Global_Sales", ascending=asc).iloc[0]
    return f"{linha['Name']} ({linha['Platform']}), {linha['Global_Sales']} milhões"


def jogos_por_publisher(pub):
    return len(df[df["Publisher"].str.lower() == pub.lower()])


def mais_vendido_regiao(region):
    coluna = {
        "NA": "NA_Sales",
        "EU": "EU_Sales",
        "JP": "JP_Sales",
    }[region]

    linha = df.sort_values(coluna, ascending=False).iloc[0]
    return f"{linha['Name']} vendeu {linha[coluna]} milhões na região {region}"


def contagem_por_coluna(col):
    return df[col].value_counts().idxmax()


def menor_contagem_por_coluna(col):
    return df[col].value_counts().idxmin()

## Definir Agente orquestrador

In [12]:
SYSTEM_PROMPT = """
Você é um assistente especialista em vendas de videogames.
Sempre responda em português brasileiro.
Não mostre raciocínios internos.
Use a ferramenta 'buscar_semelhantes' somente quando a intenção for 'buscar_semelhantes'.
"""

agent = create_agent(
    model=model,
    tools=[buscar_semelhantes],
    system_prompt=SYSTEM_PROMPT
)


def consultar(pergunta: str):
    intent = extrair_intencao(pergunta)
    i = intent["intent"]
    n = intent.get("n")
    publisher = intent.get("publisher")
    region = intent.get("region")

    # ---- INTENÇÕES COM CONSULTA EXATA ----
    if i == "mais_vendido":
        return jogo_unico(df, asc=False)

    if i == "menos_vendido":
        return jogo_unico(df, asc=True)

    if i == "top_n_mais_vendidos" and n:
        return top_n(df, n, asc=False)

    if i == "top_n_menos_vendidos" and n:
        return top_n(df, n, asc=True)

    if i == "soma_top_n_mais_vendidos" and n:
        return f"Soma: {soma_n(df, n, asc=False)} milhões"

    if i == "soma_top_n_menos_vendidos" and n:
        return f"Soma: {soma_n(df, n, asc=True)} milhões"

    if i == "quantidade_por_publisher" and publisher:
        return f"A publisher {publisher} lançou {jogos_por_publisher(publisher)} jogos."

    if i == "mais_vendido_regiao" and region:
        return mais_vendido_regiao(region)

    if i == "ano_com_mais_jogos":
        return f"Ano com mais jogos: {contagem_por_coluna('Year')}"

    if i == "ano_com_menos_jogos":
        return f"Ano com menos jogos: {menor_contagem_por_coluna('Year')}"

    if i == "plataforma_com_mais_jogos":
        return f"Plataforma com mais jogos: {contagem_por_coluna('Platform')}"

    if i == "plataforma_com_menos_jogos":
        return f"Plataforma com menos jogos: {menor_contagem_por_coluna('Platform')}"

    if i == "genero_com_mais_jogos":
        return f"Gênero com mais jogos: {contagem_por_coluna('Genre')}"

    if i == "genero_com_menos_jogos":
        return f"Gênero com menos jogos: {menor_contagem_por_coluna('Genre')}"

    # ---- FALLBACK RAG ----
    eventos = agent.stream(
        {"messages": [{"role": "user", "content": pergunta}]},
        stream_mode="values",
    )

    final = ""
    for ev in eventos:
        final = ev["messages"][-1].content

    return final

## Inicializar chat

In [14]:
if __name__ == "__main__":
    print("Agente iniciado.")
    print("Digite sair para fechar o chat.")
    while True:
        q = input("\nPergunta: ")
        if q.strip().lower() == "sair":
            break
        print("\n>>>", consultar(q))

Agente iniciado.
Digite sair para fechar o chat.

>>> Yokai Watch Busters (3DS), 2.28 milhões

>>> Ano com mais jogos: 2007.0

>>> Gênero com mais jogos: Action

>>> Os 9 jogos mais vendidos de todos os tempos são:

1. Minecraft
2. Grand Theft Auto V
3. Tetris
4. Wii Sports
5. PUBG: Battlegrounds
6. The Legend of Zelda: Breath of the Wild
7. The Elder Scrolls V: Skyrim
8. Super Mario Bros.
9. Pokémon Red, Green, Blue, and Yellow

Esses jogos são conhecidos por sua popularidade e sucesso comercial em todo o mundo.

>>>                     Name Platform  Global_Sales
              Wii Sports      Wii         82.74
       Super Mario Bros.      NES         40.24
          Mario Kart Wii      Wii         35.82
       Wii Sports Resort      Wii         33.00
Pokemon Red/Pokemon Blue       GB         31.37

>>>                      Name Platform  Global_Sales
               Wii Sports      Wii         82.74
        Super Mario Bros.      NES         40.24
           Mario Kart Wii      Wii  